### **PIP**

In [1]:
!pip install langchain-community # Install the langchain-community package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [2]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.2 MB/s eta 0:00:00


In [3]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 9.6 MB/s eta 0:00:00


### **Read**

In [4]:
import pandas as pd
from langchain.docstore.document import Document
import openpyxl
import os
from langchain_community.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [5]:
df = pd.read_excel("/content/Monthly_expence.xlsx", sheet_name="NOV_24", engine='openpyxl')
# Convert each row to a Document
documents = []
for _, row in df.iterrows():
    content = f"Date: {row['Date']}, Category: {row['Category']}, Amount: {row['Amount']}"
    documents.append(Document(page_content=content))

### **Preprocessing**

In [11]:
import textwrap
def wrap_text(text, width=100):
    lines=text.split("\n") #split text into line based new lines
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines] #wrap each line indivitually
    wrapped_text = "\n".join(wrapped_lines)
    return wrapped_text
wrap_text(str(documents))

'[Document(metadata={}, page_content=\'Date: 2024-11-01 00:00:00, Category: petrol, Amount: 200\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-03 00:00:00, Category: Room rent, Amount: 3333\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-04 00:00:00, Category: Setc, Amount: 585\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-06 00:00:00, Category: recharge, Amount: 479\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-07 00:00:00, Category: eggs, Amount: 36\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-07 00:00:00, Category: café lunch, Amount: 75\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-07 00:00:00, Category: ice cream, Amount: 30\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-07 00:00:00, Category: gift, Amount: 200\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-08 00:00:00, Category: chicken, Amount: 60\'),\nDocument(metadata={}, page_content=\'Date: 2024-11-08 00:00:00, Category: eggs, Amount: 36\'),\nDocumen

In [12]:
# from langchain.text_splitter import CharacterTextSplitter
# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# text_splitter = text_splitter.split_documents(documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
text_splitter = text_splitter.split_documents(documents)

In [15]:
print(text_splitter)

[Document(metadata={}, page_content='Date: 2024-11-01 00:00:00, Category: petrol, Amount: 200'), Document(metadata={}, page_content='Date: 2024-11-03 00:00:00, Category: Room rent, Amount: 3333'), Document(metadata={}, page_content='Date: 2024-11-04 00:00:00, Category: Setc, Amount: 585'), Document(metadata={}, page_content='Date: 2024-11-06 00:00:00, Category: recharge, Amount: 479'), Document(metadata={}, page_content='Date: 2024-11-07 00:00:00, Category: eggs, Amount: 36'), Document(metadata={}, page_content='Date: 2024-11-07 00:00:00, Category: café lunch, Amount: 75'), Document(metadata={}, page_content='Date: 2024-11-07 00:00:00, Category: ice cream, Amount: 30'), Document(metadata={}, page_content='Date: 2024-11-07 00:00:00, Category: gift, Amount: 200'), Document(metadata={}, page_content='Date: 2024-11-08 00:00:00, Category: chicken, Amount: 60'), Document(metadata={}, page_content='Date: 2024-11-08 00:00:00, Category: eggs, Amount: 36'), Document(metadata={}, page_content='Da

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings()
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from groq import Groq
db = FAISS.from_documents(text_splitter, embeddings)

<ipython-input-13-8cb94b9e4d91>:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [17]:
query = "Room rent"
doc =db.similarity_search(query)
print(doc)

[Document(metadata={}, page_content='Date: 2024-11-03 00:00:00, Category: Room rent, Amount: 3333'), Document(metadata={}, page_content='Date: 2024-11-10 00:00:00, Category: trip food, Amount: 150'), Document(metadata={}, page_content='Date: 2024-11-10 00:00:00, Category: trip food, Amount: 100'), Document(metadata={}, page_content='Date: 2024-11-09 00:00:00, Category: trip food, Amount: 160')]


### **BY GROQ**

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(k=1, memory_key='chat_history', return_messages=True)
from langchain.chains import RetrievalQA
groq_api_key = "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"
model = 'llama3-8b-8192'
groq_chat = ChatGroq(groq_api_key=groq_api_key, model_name=model)
# Initialize the QA chain with Groq model
qa_chain = load_qa_chain(groq_chat, chain_type="stuff")
# Function to get the answer
def get_answer_from_document(query):
    docresult = db.similarity_search(query)
    result = qa_chain.run(input_documents=docresult, question=query)
    return result

# Ask a question from the document
user_question = "total sum of november month amount"
response = get_answer_from_document(user_question)

print(f"Answer: {response}")

Answer: Let's add up the amounts for the grocery expenses in November:

* 2024-11-12: 111
* 2024-11-14: 104
* 2024-11-17: 132
* 2024-11-22: 119

Total sum: 111 + 104 + 132 + 119 = 466


### **BY Transformer**

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# model_name = "EleutherAI/gpt-neo-1.3B"
model_name = "EleutherAI/gpt-neo-125M"
# model_name = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
def get_answer_from_document(query):
    docresult = db.similarity_search(query)
    # Concatenate the question with the document context
    input_text = f"{query}\n{docresult}"
    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    # Generate the response
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,         # Increase max_length for a longer response
        temperature=0.8,        # Control randomness
        num_return_sequences=1,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id)  # Ensure the model knows when to stop
    # Decode and return the result
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
# Ask a question from the document
user_question = "how much room rent im spending"
response = get_answer_from_document(user_question)
print(f"Answer: {response}")

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer: how much room rent im spending
[Document(metadata={}, page_content='Date: 2024-11-03 00:00:00, Category: Room rent, Amount: 3333'), Document(metadata={}, page_content='Date: 2024-11-10 00:00:00, Category: trip food, Amount: 150'), Document(metadata={}, page_content='Date: 2024-11-09 00:00:00, Category: trip food, Amount: 160'), Document(metadata={}, page_content='Date: 2024-11-09 00:00:00, Category: trip food, Amount: 130')]
In case you have to spend the same amount of money or rent, then you should buy new furniture.

If I want to rent furniture that is similar to a bedroom with a single bed, I can use the book that I already purchased
